In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score, \
    mean_absolute_error, \
    median_absolute_error
import tensorflow as tf
import joblib

In [11]:
Path="Raw_Data/grouped_dates_monterey_avg.csv"
df=pd.read_csv(Path)

,LATITUDE,LONGITUDE,ELEVATION,EMNT,EMXT,TAVG
0,36.574080,-121.786360,64.260000,4.575000,26.125,11.800000
1,36.569425,-121.769075,71.025000,0.733333,22.200,10.833333
2,36.574080,-121.786360,64.260000,0.975000,24.175,11.900000
3,36.574080,-121.786360,64.260000,0.300000,24.750,10.025000
4,36.574080,-121.786360,64.260000,1.125000,25.275,10.675000
...,...,...,...,...,...,...
96,36.637198,-121.759970,57.528571,2.250000,28.900,13.250000
97,36.637198,-121.759970,57.528571,1.650000,20.850,12.100000
98,36.637198,-121.759970,57.528571,1.950000,23.300,11.200000
99,36.664012,-121.766689,44.466667,-1.400000,26.650,12.200000
